In [ ]:
import os
import subprocess
from pathlib import Path
import torch
from mmdet3d.evaluation.metrics import nuscenes_metric as nus_metric
from mmdet3d.evaluation.metrics.nuscenes_metric import output_to_nusc_box
import json 
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import transform_matrix
import operator
from functools import reduce
from pathlib import Path
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.eval.detection.config import config_factory
from nuscenes.eval.detection.evaluate import NuScenesEval
from nuscenes.utils.data_classes import RadarPointCloud
from classes import cls_attr_dist, class_names, mini_val_tokens
from custom_env import home_dir, output_dir, preds_dir, model_dir, is_set_to_mini
from custom_env import dataset_root as dataroot
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility

import nuscenes_render

In [ ]:
eval_set_map = {
        'v1.0-mini': 'mini_val',
        'v1.0-trainval': 'val',
        'v1.0-test': 'test'
    }

dataset_version = 'v1.0-mini' if is_set_to_mini() else 'v1.0-trainval'
try:
    eval_version = 'detection_cvpr_2019'
    eval_config = config_factory(eval_version)
except:
    eval_version = 'cvpr_2019'
    eval_config = config_factory(eval_version)
    

DETECTION_THRESHOLD = 0.35

backend_args = None
nusc = NuScenes(version=dataset_version, dataroot = dataroot)
ann_file = f'{dataroot}nuscenes_infos_val.pkl'
metric='bbox'

pcd_path = f"{dataroot}/samples/LIDAR_TOP/"
mmdet_path = f"{home_dir}/software/mmdetection3d"
pcd_list = os.listdir(pcd_path)

In [ ]:
list_of_classes = ["ped", "obs"]

PED = 0
OBS = 1
EMPTY = 2

labels = {0: "ped", 1: "obs", 2:"empty"}

conf_mat_mapping = {
    "pedestrian": PED,
    "bus": OBS,
    "car" : OBS,
    "truck": OBS,
    "bicycle": OBS,
    "motorcycle": OBS,
    "traffic_cone": OBS
}

In [ ]:
def __load_ego_veh(self, sample_token:str):
    sample = nusc.get('sample', sample_token)
    sd_record = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
    return nusc.get('ego_pose', sd_record['ego_pose_token'])

print(__load_ego_veh(nusc, "b5989651183643369174912bc5641d3b"))

In [ ]:
from nuscenes_render import convert_ego_pose_to_flat_veh_coords, render_specific_gt_and_predictions, render_sample_data_with_predictions
from generate_confusion_matrix import GenerateConfusionMatrix, convert_from_EvalBox_to_Box

generator = GenerateConfusionMatrix(nusc=nusc, 
                                    config=eval_config,
                                    eval_set=eval_set_map[dataset_version],
                                    result_path=f'{model_dir}/results_nusc.json',
                                    output_dir=os.getcwd(),
                                    verbose=True,
                                    conf_mat_mapping=conf_mat_mapping,
                                    list_of_classes=list_of_classes,
                                    distance_parametrized=True,
                                    max_dist=100,
                                    distance_bin=10)

In [ ]:
import random

sample = "b5989651183643369174912bc5641d3b"
random.seed(42)
sample = random.choice(generator.sample_tokens)
print(sample)
sample = nusc.get('sample', sample)

In [ ]:
l = []
pred_boxes = [generator.disc_pred_boxes[rad_band][sample['token']] for rad_band in generator.disc_pred_boxes.keys()]

[l.extend(box) for box in pred_boxes]



sd_record = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
cs_record = nusc.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
sensor_record = nusc.get('sensor', cs_record['sensor_token'])
pose_record = nusc.get('ego_pose', sd_record['ego_pose_token'])

print(f"---------------EGO POSE: \n{pose_record['translation']}")
print(f"\n---------------GROUND_TRUTH: ")
for box in generator.gt_boxes.all[:5]: print(box.translation)
print(f"\n---------------PREDS: ")
for box in l[:5]: print(box.translation)
# for box in generator.pred_boxes.all[:5]: print(box.translation)





In [ ]:
l = [convert_from_EvalBox_to_Box(box) for box in l]


In [ ]:
sample['data']['LIDAR_TOP'] #a977b0e797694036a9c40ab80f9d0ebc

In [ ]:

render_sample_data_with_predictions(sample_data_token=sample['data']['LIDAR_TOP'], 
                                    with_anns = True, 
                                    underlay_map = True,
                                    use_flat_vehicle_coordinates = True,
                                    pred_boxes=l,
                                    nusc=generator.nusc)

In [18]:
for annotation_token in sample['anns']:
    anntn = nusc.get('sample_annotation', annotation_token)
    print(anntn)


{'token': 'dbed4cee903546d5b402f9ace1d3f399', 'sample_token': 'f753d3f87e5b40af87ff2cbf7c8e7082', 'instance_token': 'ab8d210d019b44abae3c4e0eee6ce8ba', 'visibility_token': '4', 'attribute_tokens': ['ab83627ff28b465b85c427162dec722f'], 'translation': [631.196, 1640.849, 1.131], 'size': [0.621, 0.647, 1.778], 'rotation': [0.3248914721141039, 0.0, 0.0, 0.945751305231735], 'prev': 'decf1f76406945468e1446925c139e7c', 'next': '5b661c6079224ec1ab4d5f5ca9b47b1e', 'num_lidar_pts': 3, 'num_radar_pts': 1, 'category_name': 'human.pedestrian.adult'}
{'token': '02f4d59d74a04e95929e61b1ab328556', 'sample_token': 'f753d3f87e5b40af87ff2cbf7c8e7082', 'instance_token': '3b19bb63d0924f4da3034819357d6c8b', 'visibility_token': '4', 'attribute_tokens': ['ab83627ff28b465b85c427162dec722f'], 'translation': [605.212, 1638.044, 2.283], 'size': [0.688, 0.944, 1.904], 'rotation': [0.2980330202825458, 0.0, 0.0, 0.9545555608875073], 'prev': 'd8427cc7a2d94f9ab319c8c987ed32eb', 'next': 'a3eccc696c80448f9b7ec989be07aa2

In [ ]:
sd_record = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
sd_record['filename']

In [ ]:
nusc.get('ego_pose', sd_record['ego_pose_token'])

In [ ]:
from pyquaternion import Quaternion
sd_record = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
pose = nusc.get('ego_pose', sd_record['ego_pose_token'])
q = Quaternion(pose['rotation'])

In [ ]:
import math
math.atan2(pose['translation'][1], pose['translation'][0])

In [ ]:
q.yaw_pitch_roll

psi = q.yaw_pitch_roll[0]

psi/np.pi * 180

In [ ]:
from nuscenes.eval.common.utils import quaternion_yaw
# PREFER THIS
rotation = Quaternion(pose['rotation'])
yaw = quaternion_yaw(rotation) / np.pi * 180

yaw

In [ ]:
import random

sample = "b5989651183643369174912bc5641d3b"
sample = nusc.get('sample', sample)

nuscenes_render.render_sample_data_with_predictions(sample["data"]["LIDAR_TOP"], nusc=nusc)

# Figure out the coordinate frame
# Plot the yaw on the map below

In [ ]:
# nusc.render_sample("b5989651183643369174912bc5641d3b")

In [ ]:
scene = nusc.scene[0]
scene

In [ ]:
def get_ego_pose(token):
    for ego_pose in nusc.ego_pose:
        if ego_pose['token'] == token:
            return ego_pose
    print("Ego pose not found for token", token)
        
get_ego_pose(sample['token'])

In [ ]:
sample = "b5989651183643369174912bc5641d3b"
sample = nusc.get('sample', sample)

a, boxes, b = nusc.get_sample_data(sample["data"]["LIDAR_TOP"], 
                     box_vis_level=BoxVisibility.ANY,
                     use_flat_vehicle_coordinates=True)

np.degrees(np.arctan2(boxes[0].center[1], boxes[0].center[0]))

In [ ]:
nusc.get('ego_pose', nusc.get('sample', sample['token'])['ego_pose_token'])

In [ ]:
np.array(boxes[0].center[:2])

In [ ]:
from cluster import RadiusBand

r = RadiusBand(
                 sample_token = "a",
                 gt_boxes=[], 
                 ego_veh:{},
                 radius_band:(0, 11),
                 max_distance_bw_obj:10)